Wikipedia Retriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriver = WikipediaRetriever(top_k_results=2, lang="en")

In [3]:
query = "History of india with respect to chinese"

docs = retriver.invoke(query)

In [4]:
for i , docs in enumerate(docs):
    print(f"\n -- Result {i+1}--")
    print(f"Content: \n {docs.page_content}...")


 -- Result 1--
Content: 
 Anatomically modern humans first arrived on the Indian subcontinent between 73,000 and 55,000 years ago. The earliest known human remains in South Asia date to 30,000 years ago. Sedentariness began in South Asia around 7000 BCE; by 4500 BCE, settled life had spread, and gradually evolved into the Indus Valley Civilisation, one of three early cradles of civilisation in the Old World, which flourished between 2500 BCE and 1900 BCE in present-day Pakistan and north-western India. Early in the second millennium BCE, persistent drought caused the population of the Indus Valley to scatter from large urban centres to villages. Indo-Aryan tribes moved into the Punjab from Central Asia in several waves of migration. The Vedic Period of the Vedic people in northern India (1500–500 BCE) was marked by the composition of their extensive collections of hymns (Vedas). The social structure was loosely stratified via the varna system, incorporated into the highly evolved pres

Vector Store Retrievers

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [8]:
document=[
    Document(page_content="Langchain is good use it every day"),
    Document(page_content="Langchain is great use it every day"),
    Document(page_content="Langchain is amazing use it every day")
]

In [10]:
embedding_model = GoogleGenerativeAIEmbeddings(
    model = "gemini-embedding-001"
)
vectorStore = Chroma.from_documents(
    documents = document,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [11]:
retriver = vectorStore.as_retriever(search_kwargs ={"k":2})

In [12]:
query = "How is langchain"
results = retriver.invoke(query)

In [14]:
for i, doc in enumerate(document):
    print(f"\n --- Result {i+1}---")
    print(doc.page_content)


 --- Result 1---
Langchain is good use it every day

 --- Result 2---
Langchain is great use it every day

 --- Result 3---
Langchain is amazing use it every day


In [15]:
vectorStore.similarity_search(query, k=2)

[Document(metadata={}, page_content='Langchain is good use it every day'),
 Document(metadata={}, page_content='Langchain is great use it every day')]

MMR

In [16]:
#Sample documents
docs = [
Document(page_content="LangChain makes it easy to work with LLMs."),
Document(page_content="LangChain is used to build LLM based applications."),
Document(page_content="Chroma is used to store and search document embeddings."),
Document(page_content="Embeddings are vector representations of text."),
Document(page_content="MMR helps you get diverse results when doing similarity search."),
Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
from langchain_community.vectorstores import FAISS

embedding_model = GoogleGenerativeAIEmbeddings(model = "gemini-embedding-001")

vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [25]:
retriver = vectorstore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":3, "lambda_mult":0.5}
)

In [26]:
query = "what is Langchain"
results= retriver.invoke(query)

In [27]:
for i , doc in enumerate(results):
    print(f"\n --- Result {i+1}---")
    print(doc.page_content)


 --- Result 1---
LangChain is used to build LLM based applications.

 --- Result 2---
Embeddings are vector representations of text.

 --- Result 3---
LangChain supports Chroma, FAISS, Pinecone, and more.


Multi-Query Retriever

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from dotenv import load_dotenv
import os

In [35]:
#Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "45"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "11"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source":"I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"})
]

In [36]:
embedding_model = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

VectorStore = FAISS.from_documents(all_docs, embedding_model)

In [37]:
similarity_retriever = vectorStore.as_retriever(search_type = "similarity", search_kwargs = {"k":5})

In [46]:
load_dotenv()
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=VectorStore.as_retriever(search_kwargs={"k":5}),
    llm = ChatOpenAI(
        model = 'gpt-4o-mini',
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE")  
    )
)

In [47]:
query="how to improve energy levels and maintain balance?"

In [48]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [50]:
for i , doc in enumerate(similarity_results):
    print(f"\n --- Result {i+1}---")
    print(doc.page_content)

print("+"*150)

for i , doc in enumerate(multiquery_results):
    print(f"\n --- Result {i+1}---")
    print(doc.page_content)


 --- Result 1---
Langchain is good use it every day

 --- Result 2---
Langchain is great use it every day

 --- Result 3---
Langchain is amazing use it every day
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

 --- Result 1---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 --- Result 2---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 --- Result 3---
The solar energy system in modern homes helps balance electricity demand.

 --- Result 4---
Regular walking boosts heart health and can reduce symptoms of depression.

 --- Result 5---
Consuming leafy greens and fruits helps detox the body and improve longevity.

 --- Result 6---
Deep sleep is crucial for cellular repair and emotional regulation.


Context Compression

In [51]:
from langchain_community.vectorstores import FAISS
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [52]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [53]:
Vectorstore = FAISS.from_documents(docs, embedding_model)

In [54]:
base_retriever = Vectorstore.as_retriever(search_kwargs={"k":5})

In [55]:
llm = ChatOpenAI(
        model = 'gpt-4o-mini',
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE")  
    )
compressor = LLMChainExtractor.from_llm(llm)

In [56]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)


In [57]:
query = "What is PhotoSynthesis"
compressed_results = compression_retriever.invoke(query)

In [58]:
for i, doc in enumerate(compressed_results):
    print(f"\n, --- Result {i+1}---")
    print(doc.page_content)


, --- Result 1---
Photosynthesis is the process by which green plants convert sunlight into energy.

, --- Result 2---
Photosynthesis does not occur in animal cells.

, --- Result 3---
The chlorophyll in plant cells captures sunlight during photosynthesis.
